## Greenplum
**Greenplum** — это универсальная СУБД с массивно-параллельной архитектурой и открытым исходным кодом. Она используется для построения хранилищ данных, расчёта витрин, расширенной аналитики, поставки данных для BI-решений.

**Managed Greenplum** — сервис, который позволяет разворачивать и использовать Greenplum в облаке Yandex Cloud. Этот сервис также автоматизирует обслуживание и администрирование кластера Greenplum, а команда Yandex Cloud оказывает поддержку СУБД.

Массивно-параллельная архитектура (MPP)

А ещё я буду следить за расходом ресурсов. Чтобы не тратить лишние деньги, следуй моим рекомендациям в заданиях!
Несколько советов, которые помогут избежать ненужных трат:
 - 👉 Останавливай кластеры баз данных в перерывах между выполнением практических работ. 
 - Ты не снизишь потребление до нуля, но сократишь расходы.
 - 👉 Контролируй расходы с помощью бюджета и уведомлений, которые можно настроить в разделе Биллинг. 
 - Как это сделать, рассказывается в документации.
 - 👉 Не забудь удалить ресурсы после завершения курса. 
 - Я, конечно, ещё напомню об этом. Если тебе вдруг нужно удалить платёжный аккаунт, читай об этом в документации.

 - У Greenplum массивно-параллельная архитектура.
 - Greenplum способна хранить и быстро обрабатывать большие объёмы данных. 
 - Greenplum поддерживает различные форматы физического хранения данных в таблицах: построчно или поколоночно, со сжатием или без сжатия.
 - В основе Greenplum лежит СУБД PostgreSQL.

минусы:
 - Высокие накладные расходы на запуск запроса.
 - Операции обновления и удаления данных в Greenplum возможны, но приводят к дополнительному расходу места и необходимости периодической очистки.
 - Ограничения на конкурентность нагрузки — количество одновременно выполняемых запросов.

❗️ Учитывая эти особенности MPP, Greenplum **не нужно** выбирать для систем с преимущественной OLTP-нагрузкой — систем, где нагрузка характеризуется большим количеством «быстрых» запросов, возвращающих небольшое количество строк, а также точечным обновлением данных.**НЕ подходит** Система карточного процессинга в банке - в такой системе будет высокая транзакционная нагрузка, с которой Greenplum справляется не очень хорошо. **НЕ подходит** для Бэкенд интернет-магазина. **Не подходит** для Приём потока данных с датчиков промышленного оборудования и дальнейшая передача их в озеро данных на Hadoop.❗️ Greenplum **не подходит** в качестве системы хранения и обработки неструктурированных данных


Greenplum может хранить неструктурированные данные, но основное её назначение, как уже было сказано, — **хранение структурированных данных в таблицах и обработка их с помощью SQL**.

тест
1. К какой СУБД ближе всего SQL-синтаксис Greenplum?
 - PostgreSQL

2. С каким профилем нагрузки Greenplum справляется лучше всего? Выберите два верных ответа.
 - яжёлые и долгие запросы, обрабатывающие большие массивы данных.
Да, Greenplum хорошо работает с большими объёмами данных благодаря распараллеливанию нагрузки.
 - Запросы, использующие соединения нескольких таблиц, оконные функции и агрегации.
Да, Greenplum лучше всего справляется с аналитической нагрузкой.

3. Для каких сценариев подходит СУБД Greenplum? Выберите два верных ответа.
 - Извлечение данных из разных источников, очистка и интеграция данных.
 - Создание базы данных, объединяющей данные из разнородных СУБД и хранящей историю изменений.
Да, Greenplum подходит для обработки структурированных данных, сложных преобразований и хранения больших объёмов.


## Архитектура Greenplum
#### Массивно-параллельная архитектура (MPP)
У Greenplum массивно-параллельная архитектура. Это означает, что СУБД работает на нескольких серверах, объединённых сетью, в каждом из которых есть собственные ресурсы:
 - ⭐️ Процессор
 - ⭐️ Оперативная память
 - ⭐️ Дисковая подсистема

Такой массив серверов, объединённых внутренней сетью, называется **кластером**.

Сервера в этой системе не используют совместных аппаратных ресурсов, поэтому их производительность не зависит друг от друга.

smp = Symmetric Multiprocessing, SMP

<img src='images/00004.png' width=500><br>


⭐️ **Сходства Greenplum с PostgreSQL**:
 - Синтаксис SQL, объекты базы данных и встроенные функции.
 - Структура системного каталога.
 - Механизм реализации транзакций.
 - Ролевая модель.
 - Работа драйвера для PostgreSQL.

⭐️ **Отличия Greenplum от PostgreSQL**: 
 - У Greenplum есть иная, чем у PostgreSQL, функциональность и специфичные изменения в системном каталоге.
 - Некоторая функциональность PostgreSQL недоступна в Greenplum.
 - Все механизмы PostgreSQL в Greenplum переписаны под MPP-архитектуру.

💡 Greenplum версии 6 (в настоящий момент в облаке доступна только она) написана на базе PostgreSQL версии 9.4. Версия PostgreSQL не меняется при минорных обновлениях Greenplum.

#### Архитектура кластера Greenplum
#### Инфраструктура кластера
Кластер Greenplum в облаке всегда состоит из нескольких хостов, среди которых выделен отдельный хост, к которому подключаются клиенты, — **мастер-хост**. Мастер-хост дублируется в целях отказоустойчивости **резервным мастер-хостом**. В кластере  Managed Greenplum всегда есть ровно один мастер-хост и один резервный мастер-хост.

Все остальные хосты не допускают подключения клиентов и предназначены для хранения и обработки данных БД. Эти хосты называются **сегмент-хосты**, и их может быть несколько.

Все **хосты объединены внутренней сетью**, обеспечивающей работоспособность СУБД. Для подключения **клиентов** к базе и загрузке-выгрузке данных используется другая сеть — **внешняя**.
<img src='images/00005.png' width=500><br>

#### Программные компоненты
На каждом хосте Greenplum запущены один или несколько экземпляров PostgreSQL. Эти экземпляры называются **сегментами** Greenplum. 

В целях отказоустойчивости все сегменты в кластере дублируются, то есть у каждой порции данных в Greenplum есть **копия (реплика)** на другом хосте кластера.

##### Типы сегментов Greenplum: 
⭐️ **Мастер (Master / coordinator)**

Мастер присутствует в кластере в единственном экземпляре, резервируется standby мастером и располагается на мастер-хосте.  Он принимает подключения клиентов, выдаёт результаты запросов, обеспечивает координацию работы всего кластера.

⭐️ **Резервный мастер (Standby)**. 
Принимает поток репликации с мастера. Располагается на отдельном резервном мастер-хосте. При нормальной работе к нему невозможно подключиться, но в случае отказа мастера можно активировать резервный мастер и перевести его в режим мастера.

⭐️ **Первичные сегменты (Primary segments)**. 
Этих сегментов в кластере несколько. Они располагаются на сегмент-хостах. В своих директориях первичные сегменты хранят пользовательские данные и реплику каталога БД. На первичных сегментах обрабатываются SQL- и DML-операции, они несут основную пользовательскую нагрузку.

⭐️ **Сегменты-зеркала (Mirror segments)**. 
Этих сегментов в кластере столько же, сколько и первичных. Зеркала располагаются на сегмент-хостах и хранят реплики данных с первичных сегментов. Они не обрабатывают запросы, а только принимают поток репликации данных, поэтому почти не создают нагрузку на процессор и память. Репликация на зеркалах синхронная: при записи данных на первичный сегмент транзакция не закончится, пока данные не реплицируются на зеркало.

<img src='images/00009.png' width=500><br>

Есть **сегмент** — программный компонент СУБД, а есть **сегмент-хост** — элемент инфраструктуры (в облаке это — виртуальная машина). Как правило, когда говорят «сегмент», по умолчанию подразумевают первичный сегмент Greenplum. Так, в диаграмме выше в кластере сконфигурировано четыре сегмента на хост.

Сегмент-хост, как правило, содержит в себе несколько сегментов Greenplum. А мастер-хост всегда содержит только один мастер.

**пример**

Представь, что у нас есть кластер Greenplum, состоящий из 10 серверов. Один сервер — мастер, ещё один сервер — реплика мастера и 8 сегментных серверов. При инициализации кластера мы сконфигурировали 4 сегмента на хост. Получается, у нас есть 8 сегментных серверов x 4 сегмента на хост = 32 сегмента. Помимо этих 32 сегментов у нас обязательно есть 32 зеркала, располагающихся на тех же хостах, а также сегменты мастера и резервного мастера. 
Всего в кластере фактически 32+32+1+1 = 66 сегментов. Их можно увидеть в конфигурационной таблице gp_segment_configuration. Однако данные обрабатываются на 32 первичных сегментах. В этом случае мы говорим, что у нас в кластере 32 сегмента, оставляя зеркала и мастера за скобками как сами собой разумеющиеся.

💡 У одного первичного сегмента одно и только одно зеркало, которое располагается на другом сегмент-хосте. Количество зеркал совпадает с количеством первичных сегментов.

💡 Управление кластером Greenplum и подключение клиентов всегда осуществляется через мастер-хост. Подключающийся пользователь видит только мастер, кластер Greenplum для пользователя выглядит как одна база данных PostgreSQL.

Первичные сегменты и зеркала равномерно распределены по сегмент-хостам. Это связано с тем, что зеркала почти не утилизируют процессор и память, и если их вынести на отдельный хост, он будет недозагружен.

можно посмотреть список всех сегментов вместе с мастером и резервным мастером в системной таблице gp_segment_configuration. `select * from gp_segment_configuration;`


#### Принципы параллельной обработки в Greenplum
Выполнение запросов производится на первичных сегментах. Всегда задействуются все сегменты — пользователь не может задать выполнение запроса на части сегментов.

Сегменты Greenplum работают **синхронно**, обеспечивая параллелизацию обработки каждого запроса. Каждый запрос распараллеливается на столько процессов, сколько есть первичных сегментов в кластере.

💡 При выполнении запроса важно обеспечить равномерность нагрузки на сегменты. Запрос считается выполненным и результат возвращается пользователю только после того, как отработали все сегменты. Самый долго работающий сегмент будет определять время работы запроса в целом.

Ситуация, когда количество данных, обрабатываемых запросом на разных сегментах, сильно отличается, называется **перекос** (в англоязычной документации - **skew**).

<img src='images/00010.png' width=400><br>

Со стороны клиента параллельная загрузка данных в Greenplum выглядит как **SELECT** из таблицы, которая ссылается на внешний источник, а выгрузка данных из Greenplum — как **INSERT** в неё.<br>
💡 Такие **таблицы для загрузки называются внешними** и не содержат данных, но содержат путь до данных во внешнем источнике в своём DDL.

Под капотом этого механизма используются **сервисы PXF или gpfdist**, входящие в состав Greenplum.

**!!!вы только что прочитали самый важный урок курса. Для лёгкого прохождения обучения тебе нужно усвоить все принципы и терминологию из него.!!!**

### 📘 тест

1. В какой ситуации Greenplum гарантирует работоспособность кластера и доступность данных?
   - Отказ одного хоста. Если откажет мастер-хост, его работу сможет выполнять резервный мастер, а если откажет сегментный хост, то на другом хосте обязательно будут зеркала для всех сегментов отказавшего хоста.

2. Выберите верные утверждения об архитектуре Greenplum. Выберите три верных ответа
   - Все сегмент-хосты должны содержать одинаковое количество первичных сегментов. это необходимо для обеспечения равномерной нагрузки по сегментам.
   - Все сегмент-хосты должны содержать одинаковое количество зеркал. это необходимо для того, чтобы утилизация дискового пространства на сегментах была равномерной.
   - Пара из первичного сегмента и его зеркала всегда размещается на разных хостах. в этом случае при отказе хоста роль первичного сегмента возьмёт на себя зеркало, располагающееся на другом хосте.

3. Что такое сегмент в Greenplum?
   - Экземпляр PostgreSQL на хосте Greenplum. Сегмент в Greenplum — это приложение на базе PostgreSQL, обеспечивающее хранение данных и выполнение запросов.

4. Выберите три **неверных** утверждения о Greenplum.

  - ❌ Возможно задать набор сегментов, на которых будет выполняться запрос.
Да, это неверное утверждение. Запрос в Greenplum всегда выполняется на всех сегментах.
  - ❌Запросы в Greenplum сначала вытягивают данные с сегментов на мастер-хост и затем выполняются на мастере. Да, это неверное утверждение. Запросы в Greenplum выполняются на сегментах. Именно благодаря этому обеспечивается быстродействие, нет «бутылочного горлышка» в виде мастера.
 - ❌При выполнении запросов данные могут читаться с зеркал для снижения нагрузки на первичные сегменты. Действительно, это неверное утверждение. Зеркала используются только для принятия потока реплицированных данных с сегментов, но сами не участвуют в обработке запросов.

## 👉👈 Инфраструктура, требуемая для Greenplum
СУБД Greenplum — это программное решение, которое можно установить на любую инфраструктуру, удовлетворяющую минимальным требованиям:
 - ⚙️ процессор, совместимый с архитектурой x86_64;
 - ⚙️ 10 Gbit сеть;
 - ⚙️ операционная система Linux актуальных версий, поддерживаются дистрибутивы Centos, Ubuntu, RHEL, Oracle Linux;
 - ⚙️ минимум 16 GB оперативной памяти на сервере;
 - ⚙️ минимум 0,5 GB места на жёстком диске для каждого сегмента.

Greenplum можно установить как непосредственно на сервер, так и на виртуальную инфраструктуру — отличий в функционировании СУБД не будет. Минимальный кластер Greenplum можно развернуть даже на ноутбуке. В случае разворачивания на серверах  инсталляцию называют **«on-premise-инсталляция».** (т.е. локальная)

В случае размещения на виртуальной инфраструктуре возможны следующие варианты:
 - создание виртуальных машин и разворачивание Greenplum на них так же, как на серверах;
 - запуск управляемого сервиса Greenplum в облаке.

О втором способе и пойдёт речь в этом курсе.
### Управляемая Greenplum в Yandex Cloud

❗️ При этом есть ограничения:
 - нет возможности подключиться к серверам Greenplum напрямую по SSH;
 - нет прав суперпользователя (superuser) внутри СУБД.
Эти ограничения нужны, чтобы администратор не мог повлиять на стабильность сервиса управляемой БД, за который отвечает облако.

Специально для администраторов БД в управляемой Greenplum создана роль mdb_admin c расширенными привилегиями. Они позволяют управлять доступом к таблицам, добавлять пользователей, создавать и удалять базы данных и раздавать права.

💡 С точки зрения обычного пользователя разницы между Managed Greenplum и on-premises Greenplum нет.

Термин **Управляемая СУБД** означает, что развёртывание виртуальных машин, настройка виртуальной сети, установка СУБД, операции обслуживания, апгрейд и поддержка осуществляются облачным провайдером.

Базовые задачи, которые требуют доступа к хостам, автоматизированы облаком. Для их запуска есть соответствующий веб-интерфейс, а также можно использовать консольную утилиту для управления ресурсами облака — yc

<img src='images/00011.png' width=600><br>
💡 Некоторые действия всё равно остаются на стороне клиента облака:
 - планирование ресурсов кластера;
 - мониторинг производительности;
 - оптимизация пользовательских процессов;
 - аутентификация и авторизация пользователей;
 - разграничение доступов в базе данных.

#### тест
1. Что такое управляемая Greenplum?
 - Кластер Greenplum, развёрнутый на виртуальных машинах в облаке, для которых настроена автоматизация инсталляции и администрирования.

2. Что такое mdb_admin?
 - Роль в Greenplum для администрирования с учётом ограничений управляемого сервиса.
mdb_admin — это специальная роль в Greenplum, предназначенная для администратора БД и предоставляющая расширенные права.

3. Какие ограничения накладываются на управляемую Greenplum в Yandex Cloud? Выберите четыре верных ответа.
   - Нет доступа к серверам по SSH.Правильно, возможности подключиться к серверам Greenplum напрямую по SSH нет.
   - Нет root-доступа к операционной системе.К операционной системе серверов управляемой Greenplum не предоставляется доступ.
   - Нельзя войти в СУБД под суперпользователем.Привычная роль суперпользователя в Greenplum недоступна.
   - Нельзя создать суперпользователя. Привычная роль суперпользователя в Greenplum недоступна.

### Практика: создаём кластер
<img src='images/00012.png' width=300><br>

✔️‍️ ✔️ ✔❌❌♻️ не создавала! пропустила, т.к. нужно оплачивать яндекс облако . пока смотрю дальше. потом решу!

#### Особенности Managed решения в Yandex Cloud → Урок 1/4

## Конфигурация кластера
#### Как ресурсы Greenplum связаны друг с другом
Основная сущность, которой оперирует Managed Service for Greenplum, — кластер баз данных.<br>
💡**Кластер** — группа серверов, объединённых высокоскоростными каналами связи, представляющая с точки зрения пользователя единый аппаратный ресурс.

Каждый кластер состоит из хостов баз данных — виртуальных машин с развёрнутыми серверами СУБД. Кластер Managed Service for Greenplum включает:
 - ⭐️ Два мастер-хоста: первичный (PRIMARY) и резервный (STANDBY), который нужен для отказоустойчивости кластера.
 - ⭐️ Два или больше сегмент-хостов, на которых хранятся и обрабатываются данные. <br>
 В кластере их должно быть как минимум два, так как данные обязательно должны реплицироваться. Каждый сервер содержит свою часть данных и выполняет операции по обработке запросов.<br>
На сегмент-хостах развёрнуты зависимые СУБД (называемые в Greenplum сегментами), которые хранят части данных и исполняют большинство операций по обработке запросов. Каждый сегмент в кластере имеет одну реплику — зеркальный сегмент, который находится на другом хосте и хранит копию данных с основного сегмента. Это значит, что если с одним из сегментов что-то произойдёт, его данные всё равно сохранятся благодаря зеркальному сегменту.
#### Планирование и расчёт конфигурации кластера
##### Важнейшие параметры кластера Greenplum:
 - Количество сегмент-хостов. Это основные узлы, на которых хранятся и обрабатываются данные.
 - Количество ядер CPU. Они влияют на вычислительную мощность хоста.
 - Количество RAM. Важно для обработки запросов и хранения данных в памяти.
 - Общий размер дисков на сегмент-хостах. Определяет, сколько данных можно хранить.
 - Количество сегментов Greenplum на одном сегмент-хосте. Влияет на параллельную обработку данных.

💡 Чем больше ожидается пользователей с конкурентными запросами и чем больше хостов в кластере, тем больше должны быть CPU и RAM на мастер-хосте.

### 👩🏻‍💻📓✍🏻💡💡 Рекомендации по конфигурации кластера:
 - 🤝Для сред **разработки и тестирования** допустимо выбирать для мастера и сегментов виртуальные машины типа **Standard**.
 - 🤝В случае построения **продуктивного кластера** рекомендуется выбрать **виртуальные машины IO-optimized**. Они развивают большую производительность за счёт использования дополнительных ядер вне ВМ для обеспечения работы сети.
 - 🤝На **мастер-хосте** рекомендуется использовать **Network SSD** — быстрые сетевые реплицированные диски.
 - 🤝На **сегментных хостах в продуктивных среда**х рекомендуется использовать самые быстрые типы дисков — **Non-replicated SSD** или **Local SSD**. Последний будет быстрее, но максимальный размер Local SSD **ограничен 2,9 ТБ**.
 - 🤝Конфигурация **мастер-хоста**, в том числе **CPU и RAM**, зависит от количества пользователей базы, их профиля нагрузки и иногда от конкретных запросов, нагружающих мастер (такие могут появляться в редких случаях). При начале работы хранилища допустимо использовать минимальную конфигурацию мастер-хоста, увеличивая его мощность в случае нехватки ресурсов.

### 👩🏻‍💻📓✍🏻💡💡 Расчёт параметров для настройки кластера
#### Расчёт дискового пространства на сегмент-хостах 
следует производить по планируемому объёму данных. Главный вопрос, на который необходимо ответить при конфигурировании кластера: «Сколько несжатых данных планируется хранить в Greenplum?»

На расчёт влияют и другие факторы. 

При планировании дискового пространства рекомендуется:
 - Умножать на 2 запланированный объём данных
Данные дублируются для повышения надёжности.
 - Учитывать сжатие данных
Greenplum может сжимать данные, так что они могут занимать в 3–4 раза меньше места. Это значит, что реально требуемое на диске пространство может быть гораздо меньше первоначального объёма данных.
 - Не заполнять диски более чем на 70%
Дополнительное дисковое пространство необходимо для создания временных файлов при функционировании БД. Его также нужно предусмотреть.

**`💡 Рекомендуемый размер хранилища для сегмент-хостов можно рассчитать по формуле:<объём данных без сжатия> × 1,5`**

Формула справедлива при использовании **стандартного сжатия (zstd) первого уровня**. *(Это базовый уровень сжатия, который помогает уменьшить размер данных, сохраняя при этом хорошее соотношение между эффективностью сжатия и скоростью обработки данных)*. Она учитывает зеркалирование данных, пространство под временные файлы при выполнении тяжёлых запросов и свободное пространство, необходимое СУБД.



*Благодаря тому, что твой кластер находится в облаке, всегда можно подстроиться под изменения. Если вдруг окажется, что места на дисках не хватает, ты можешь легко увеличить объём используемого хранилища. Гибкость — одно из преимуществ использования облачных сервисов.*

**`Рекомендуемый объём CPU и RAM рассчитывается с помощью следующих формул:`**
- 💡 Суммарный объём оперативной памяти сегмент-хостов: <br>
`<объём данных без сжатия> ÷ 8 `<br>
Например, если вы планируете хранить 80 ГБ данных, потребуется 10 ГБ RAM: `80 ÷ 8 = 10`. <br>
- 💡 Суммарное количество ядер vCPU сегмент-хостов: <br>
`<объём данных без сжатия> ÷ 80`<br>
Для 80 ГБ данных потребуется 1 ядро CPU: `80 ÷ 80 = 1`.<br>
Утилизация CPU и RAM может варьироваться в зависимости от профиля нагрузки на СУБД. Главные факторы здесь — **конкурентность запросов и размер активно обрабатываемых данных**. <br>
#### Количество первичных сегментов на хост 
При настройке кластера в Greenplum вы можете выбирать, сколько первичных сегментов (основных единиц обработки данных) разместить на каждом хосте (сервере) и сколько всего хостов будет в кластере. **Общее количество сегментов в кластере задаёт степень распараллеливания всех процессов в Greenplum**. Иными словами, *число сегментов определяет, сколько запросов к базе данных будет выполняться одновременно*.

*По сути, чем больше у тебя сегментов, тем больше процессов будет работать одновременно для обработки каждого запроса. Это может ускорить выполнение отдельного запроса. Однако есть и обратная сторона: если у тебя очень много запросов одновременно (высокая конкурентность), слишком большое количество процессов может замедлить работу системы, так как эти процессы будут «мешать» друг другу.*

От выбора правильного количества сегментов зависит важный баланс между скоростью обработки запросов и общей производительностью системы.

#### Рекомендуемое количество сегмент-хостов в кластере
💡 Рекомендуется создавать больше сегмент-хостов с меньшим количеством ресурсов на каждом хосте, если их суммарные характеристики не ниже рассчитанных значений. Это поможет минимизировать последствия при выходе одного хоста из строя.

Количество сегментов на хост, выбранное при создании кластера, в дальнейшем может быть увеличено при расширении кластера. Однако расширение — достаточно длительная операция, потребляющая много ресурсов и требующая окна обслуживания, поэтому необходимо сразу рассчитывать оптимальное количество сегментов на хост. 

Рекомендуемое количество сегментов на хост зависит от двух параметров:<br>
1️⃣ Количество ядер vCPU на хосте
 - `8 ≤ vCPU < 32 — 4 сегмента`
 - `32 ≤ vCPU < 64 — 4–8 сегментов`
 - `vCPU ≥ 64 — 8–16 сегментов`
В кластере Greenplum каждому сегменту выделяется `от 2 до 16 виртуальных процессорных ядер (vCPU)`, но реплики этих сегментов используются только для записи данных на диск и не участвуют в обработке данных. Они не используют vCPU и оперативную память.

Большее число сегментов помогает быстрее обработать длинные запросы, так как они обрабатываются параллельно. Если сегментов меньше, остаётся больше ресурсов для каждого из них. Это полезно для запросов, требующих много памяти, или для обработки многих запросов одновременно.

При **высокой ожидаемой конкурентности** — от нескольких десятков до сотни одновременно выполняющихся запросов — выбирайте количество сегментов на хост ближе **к нижнему пределу**.<br>
2️⃣ Количество файлов в БД
Этим параметром часто можно пренебречь и рассчитывать его с большими погрешностями.<br>
📚 Рекомендации по количеству файлов на хосте

- Количество файлов на хосте не должно превышать 100 000:<br>
`<количество сегментов на хост> ≤ 100 000 ÷ <количество файлов в БД>`
- Количество файлов можно грубо оценить по количеству таблиц в базе данных: на каждую таблицу типа HEAP приходится как минимум 1 файл, а на таблицу, оптимизированную для добавления (APPEND-OPTIMIZED), — около 7. При этом для партицированных таблиц каждая партиция рассматривается как отдельная таблица, а для колоночно-ориентированных каждая колонка создаёт отдельный файл. Кроме того, файлы создаются для индексов.

### 📢 Пример расчёта продуктивного кластера
Для примера расчёта возьмём кейс создания продуктивного кластера. Мы хотим хранить 20 ТБ несжатых данных в Greenplum, на основе которых будут строиться витрины данных (это специальные структуры, которые помогают анализировать и визуализировать информацию). 
Планируется единоразово наполнить хранилище из источников, далее догружать каждый день новые данные, достраивая инкрементально витрины данных. Количество ожидаемых таблиц (с учётом партицирования) — 1700 APPEND-OPTIMIZED-таблиц. После расчёта витрин данные переносятся в ClickHouse, на которую будет смотреть BI-система для визуализации данных.

##### 📢 Выбор мастер-хоста
 - 1️⃣ Выбираем Network SSD — быстрые сетевые диски.
 - 2️⃣ Размер дисков, vCPU и RAM выбираем минимальный, так как не ожидается большой пользовательской нагрузки из-за использования ClickHouse как источника данных для BI.
 - 3️⃣ Тип виртуальной машины — IO-optimized, так как строим продуктивный кластер.
##### 📢 Выбор конфигурации мастер-хоста Greenplum
 - Для наших требований подходит i3-c16-m128 с Network SSD на 100 ГБ.
##### 📢Расчёт сегмент-хостов
 - Для хранения и обработки 20 480 ГБ (20 ТБ) несжатых данных понадобится кластер со следующими характеристиками сегмент-хостов:
 - Размер хранилища: 20 480 ГБ × 1,5 = 30 720 ГБ (<объем данных без сжатия> × 1,5).
 - Объём оперативной памяти: 20 480 ГБ ÷ 8 = 2 560 ГБ (<объем данных без сжатия> ÷ 8).
 - Количество ядер vCPU: 20 480 ГБ ÷ 80 = 256 (<объем данных без сжатия> ÷ 80).
Это соответствует, например, двадцати хостам i2.2xlarge (16 vCPU, 128 ГБ) с дисками 1 536‬ ГБ.
##### 📢 Расчёт количества сегментов на хост
В формуле участвует количество ядер на один хост. В нашем случае vCPU — 16, что соответствует выбору 4 сегментов: 8 ≤ vCPU < 32 — 4 сегмента.

#### тест
1. Какое минимальное количество сегмент-хостов необходимо добавить при расширении кластера Greenplum?
   - Два сегмент-хоста. Например, если уже есть 4 сегмент-хоста, добавить можно 4, но минимально требуются 2.

<img src='images/00013.png' width=500><br>

### Управление подключениями и аутентификацией
тест

1. Какой менеджер подключений используется в Managed Service for Greenplum?
 - Odyssey. Верно! Именно из-за него мы подключаемся к порту 6432 и не знаем проблем с менеджментом соединений.

2. Как добавить правило в pg_hba.conf в Managed Service for Greenplum?
 - Использовать раздел Аутентификация пользователей. Верно! Этот раздел интерфейса предоставляет удобный способ для добавления или изменения правил аутентификации, обеспечивая тонкую настройку доступа к вашему кластеру.

### Управление кластером в облаке и техническое обслуживание
##### Типы резервных копий
 - Полные резервные копии. Создаются первой и каждой седьмой копией, включают в себя полный набор данных всех баз в кластере.
 - Инкрементные копии. Сохраняется только разница с предыдущей полной или инкрементной копией, что способствует оптимальному использованию хранилища.
 - 
Резервные копии формируются автоматически один раз в сутки. Вы не можете создать резервную копию вручную или отключить этот процесс. Однако при создании или изменении кластера можно задать время начала резервного копирования.

❗ Восстановить кластер можно только в новый созданный кластер. При этом всегда восстанавливаются все таблицы во всех базах данных. Частичное восстановление кластера на данный момент не реализовано. Конфигурацию кластера, в который будет восстановлена резервная копия, необязательно делать такой же, как и у основного кластера. Однако суммарное количество сегментов нового и старого кластера должно совпадать.

##### Выбор класса хостов

Класс хостов задаёт вычислительные ресурсы, доступные каждому хосту в кластере. После создания кластера изменить выбранную платформу хостов не представляется возможным; в выпадающем списке при попытке изменения будет доступен только изначально выбранный вариант.


##### Выбор платформы хостов при создании кластера
Простейший способ масштабирования кластера — это добавление новых сегмент-хостов. Это позволяет увеличить как вычислительные мощности, так и доступный объём хранилища:


##### Расширение кластера в Managed Service for Greenplum
При расширении кластера необходимо добавлять чётное количество сегмент-хостов, так как данные должны зеркалироваться:


##### Добавление сегмент-хостов при масштабировании кластера
Если вы столкнулись с нехваткой места для хранения, но вычислительные ресурсы ещё не исчерпаны, возможно увеличение только объёма дисков. Однако следует учитывать ограничения на максимальный размер диска, определённые для различных типов дисков в Yandex Cloud. Если требуемое количество дополнительного пространства больше, то придётся добавлять новые сегмент-хосты.


##### Увеличение объёма дисков в Managed Service for Greenplum
Изменение типа хоста
Если необходима балансировка вычислительных ресурсов, то для улучшения производительности может понадобиться изменение доступных CPU и RAM. Можно в любой момент изменить тип хоста, чтобы этих ресурсов стало больше, или, наоборот, в случае избытка уменьшить их. Это делается в разделе настроек кластера
#####  тест
1. Каким образом возможно перенести регламентное техническое обслуживание кластера на другое время?
   - Сделать перенос самостоятельно на главной вкладке Обзор кластера. Верно! Именно здесь вы можете легко перенести запланированное техническое обслуживание кластера на более удобное для вас время.
2. На какой момент времени можно восстановить кластер из резервной копии?
   - На любой момент времени в пределах хранения резервных копий.Верно! Восстановить кластер можно на любой момент времени в пределах хранения резервных копий.
  
#### Ключевые тезисы модуля
##### 🍀 В этом модуле вы 🍀:
 - узнали роль и функции хостов баз данных, мастер-хостов и сегмент-хостов в кластере Greenplum;
 - научились рассчитывать параметры для конфигурации кластера, исходя из объёма данных и типа дисков;
 - познакомились с менеджером подключений Odyssey для оптимизации взаимодействия с базами данных Greenplum и основами его настройки;
 - научились настраивать правила аутентификации, обеспечивая контролируемый доступ к базам данных;
 - познакомились с основными процессами технического обслуживания кластера Greenplum в Yandex Cloud, включая автоматическое обновление, планирование обслуживания и уведомления о предстоящих операциях;
 - научились управлять процессом резервного копирования и восстановления кластера из резервной копии в новый кластер.

#### Тема 1/1: Физическая модель данных и DDL → Урок 1/8

### Ключевые сущности Greenplum

##### Что такое инстанс Greenplum, база данных, схемы, объекты и пользователи
⭐️ Инстанс Greenplum, — это набор процессов, выполняющихся на кластере и обеспечивающих возможность работать с базами данных.<br>
Эти процессы принимают подключения, выполняют запись данных на диск, обработку данных и системные задачи. Работающий кластер всегда представлен одним инстансом.<br>
⭐️ База данных (БД) — это набор таблиц с данными, представлений, функций и других объектов.
На одном инстансе Greenplum можно создать несколько БД, они могут различаться настройками. Настройки доступа пользователей для разных БД в рамках одного инстанса также могут отличаться. Кроме того, объекты БД в разных базах данных изолированы друг от друга. Обратиться из одной БД к таблице или иному объекту другой БД невозможно.<br>
Синтаксис команды для создания БД:<br>
`CREATE DATABASE database_name;`
⭐️ Схема — это пространство имён, которое содержит именованные объекты: таблицы, представления, типы данных, функции и операторы. <br>
Синтаксис команды для создания схемы:<br>
`CREATE SCHEMA schema_name`<br>
`CREATE SCHEMA IF NOT EXISTS schema_name `<br>
⭐️ Роль — это сущность, которая может владеть объектами базы данных и иметь её привилегии. Роль может рассматриваться как пользователь, группа или оба в зависимости от того, как она используется.<br>
💡 В Greenplum роли отличаются от пользователей операционной системы и являются глобальными для всего кластера (не для отдельной БД).<br>
##### Чувствительность к регистру в именах<br>
❗️ В Greenplum имена таблиц и схем не чувствительны к регистру.<br>
Поэтому, например, эти запросы будут обращаться к одной и той же таблице:<br>
```
SELECT * FROM my_table;
SELECT * FROM MY_TABLE;
```
❗️ Имена, заключённые в двойные кавычки, будут чувствительны к регистру.<br>
Эти запросы будут обращаться к разным таблицам:<br>
```
SELECT * FROM "my_table";
SELECT * FROM "MY_TABLE"; 
```
Имена таблиц, если не заключены в двойные кавычки при создании, сохраняются внутри Greenplum в нижнем регистре, то есть my_table, My_TabLE и MY_TABLE эквивалентны “my_table”.<br>
💡 Если хотите сохранить чувствительность к регистру, заключайте имена таблиц в двойные кавычки.<br>
Кавычки могут применяться для задания спецсимволов в именах: например, русских букв или знаков препинания. <br>
Аналогичные правила распространяются на имена схем, полей таблиц, функций, ролей и прочих объектов в Greenplum.<br>

##### Как взаимодействуют сущности. Неочевидные особенности Greenplum
💡 Имена объектов в разных схемах могут совпадать. В рамках одной схемы имена должны быть разными.
Существует два способа доступа к объектам:
 - Уточнение имён объектов с помощью имени схемы в качестве префикса.
 - Задание пути поиска, включающего нужные схемы или схему.<br>
💡 Путь поиска (search_path) — это специальный параметр сессии. В нём указана последовательность схем, в которых нужно искать объект, если вы не указали схему явно.<br>

Как обратиться к объекту, указав его название и местонахождение<br>
Обращаться к объекту нужно с помощью указания БД и схемы, в которой находится объект:

`SELECT * FROM database_name.schema_name.table_name; `<br>
💡 В Greenplum нельзя обращаться к объектам другой БД, поэтому **database_name** можно опустить.
Если же имя объекта чувствительно к регистру, то имя необходимо заключить в двойные кавычки:
```
SELECT * FROM schema_name."table_name";

SELECT * FROM "schema_name"."table_name";
```
❌ Некорректно заключать в кавычки и схему, и название таблицы, потому что интерпретатор выполнит поиск объекта с точкой в имени вместо нужного вам в заданной схеме: 

```--неправильный вариант обращения к таблице schema_name.table_name
SELECT * FROM "schema_name.table_name"; 
```
#### Настройка search_path
Команда CREATE TABLE при указании имени таблицы без схемы создаёт объект в текущей схеме. Текущую схему можно посмотреть с помощью функции current_schema:
```
SELECT current_schema;
```
Текущую схему можно изменить командой SET:
```
SET search_path TO schema_name1 [, schema_name2, ...]
```
💡 Этот запрос меняет настройку только в текущей сессии и не влияет на другие сессии и на других пользователей.

Можно вернуть первоначальное состояние search_path инструкцией RESET.

💡 В search_path можно задать несколько схем. Тогда функция current_schema вернёт первую существующую схему, в которой пользователю можно создавать объекты.

А при запросах к таблице без указания схемы таблица будет искаться во всех схемах из search_path по порядку списка.

### ✍️ Практическое задание 1: первоначальное состояние search_path
```
1️⃣ Создайте новую схему foo.
Код-подсказка
CREATE SCHEMA foo; 
2️⃣ Создайте три таблицы. Две таблицы с одинаковым именем bar: первую — без указания схемы, вторую — с указанием схемы. Третью — с именем baz с указанием схемы. И вставьте в них данные.
Код-подсказка
CREATE TABLE bar (id INT);
INSERT INTO bar VALUES (1);

CREATE TABLE foo.bar (id INT);
INSERT INTO foo.bar VALUES (2);

CREATE TABLE foo.baz (id INT);
INSERT INTO foo.baz VALUES (3); 
Вы создали две таблицы с одинаковым именем, но в разных схемах. Первая таблица bar создана в схеме public, так как она возвращается функцией current_schema, а вы пока не меняли путь поиска. Вторая таблица bar и таблица baz созданы в схеме foo — это было указано явно. 
3️⃣ Задайте несколько схем — some_schema (несуществующую схему), public и ранее созданную foo. Затем проверьте, какая схема текущая.
Код-подсказка
SET search_path TO some_schema, PUBLIC, foo; 
SELECT current_schema;  
Отобразилась схема public, some_schema проигнорировалась, так как её не существует.
4️⃣ Запустите выборки из таблиц — из bar, которая находится в схеме public, и из baz, находящейся в foo.
Код-подсказка
SELECT * FROM bar; 
SELECT * FROM baz;  
Отлично, всё работает!
5️⃣ Далее измените настройку search_path — в первую очередь поставьте схему foo и проверьте, изменилась ли current_schema.
Код-подсказка
SET search_path TO foo, PUBLIC; 
SELECT current_schema;   
Да, current_schema изменилась — теперь это foo.
6️⃣ Сделайте выборку из таблицы bar, которая находится в этой схеме.
Код-подсказка
SELECT * FROM bar;  
В результате вы получили строки из таблицы foo.bar.
7️⃣ Теперь сбросьте переменную search_path и проверьте, что вернулось её значение по умолчанию. Затем сделайте выборку из таблицы, находящейся в схеме foo.
Код-подсказка
RESET search_path; 
SHOW search_path; 
SELECT * FROM baz;   
На последнем запросе вы получите ошибку и сообщение о том, что таблицы не существует. Так и должно быть, ведь в search_path не указана схема foo:

ERROR: relation "baz" does not exist 
Итак, при создании таблицы без указания схемы эта таблица будет создана в схеме, которая вернётся функцией current_schema. 
При обращении к таблице без указания схемы поиск таблицы производится в каждой схеме, указанной в search_path. Возвращёна будет таблица, найденная первой при просмотре схем по порядку из search_path.
```
*Гриша
Первая практическая работа в этом модуле подошла концу, и кластер не понадобится до следующей практики. Не забудь остановить свой кластер баз данных, это поможет сократить расходы.*
#### Системные схемы
В каждой базе данных Greenplum есть несколько системных схем.

Системные схемы, содержащие полезную информацию и инструменты для разработчиков и администраторов
```
1️⃣ pg_catalog 
Основной источник знаний об объектах БД. Содержит таблицы, в которых хранятся метаданные таблиц, представлений, столбцов и остальных объектов БД. Эта схема также называется каталогом Greenplum. В каталоге содержится ряд полезных представлений. Схема pg_catalog всегда неявно присутствует в пути поиска (search_path), поэтому к её таблицам и представлениям можно обращаться без указания схемы. 
О системных схемах подробнее расскажем в четвёртом модуле.
2️⃣ information_schema 
Содержит набор удобных представлений над таблицами каталога с информацией об объектах БД. Структура этих представлений соответствует стандарту ANSI и может быть удобна для тех, кто переходит на Greenplum с другой СУБД.
3️⃣ gp_toolkit 
Содержит представления и функции для мониторинга и администрирования.
4️⃣ mdb_toolkit 
Содержит представления и функции для мониторинга и администрирования с учётом ограничений облака. Некоторые системные функции из pg_catalog или gp_toolkit требуют для запуска права суперпользователя, которых нет в управляемой Greenplum. Для этих функций есть аналог в mdb_toolkit.
Эта схема присутствует только в управляемой Greenplum.
Системные схемы, в которых Greenplum хранит внутренние вспомогательные данные
Запросы напрямую к объектам в этих схемах невозможны.
1️⃣ pg_aoseg 
Хранит системные данные для APPEND-OPTIMIZED-таблиц. Подробнее об этих таблицах вы узнаете в следующем уроке. 
2️⃣ pg_toast 
Хранит значения из других таблиц, размер которых превышает размер блока данных для таблицы (по умолчанию — 32 КБ).
3️⃣ pg_bitmapindex 
Содержит служебную информацию о bitmap-индексах.
4️⃣ pg_temp_<sess_id>
Содержит временные таблицы и другие объекты, созданные с указанием слова TEMPORARY. Для каждой пользовательской сессии создается своя схема pg_temp. Подробнее о временных таблицах вы узнаете в следующем уроке.
```

тест<br>
1. Какие варианты названия таблицы эквивалентны названию apples? Выберите четыре верных ответа.
Правильный ответ   
“apples”  Apples   apples  APPLES
2. В какой схеме будет создана таблица bar?
```
CREATE SCHEMA some_schema;
SET search_path TO foo, public, some_schema;
CREATE TABLE bar (id INT); 
```
public. Таблица будет создана в первой доступной схеме в пути поиска. Первой схемой указана foo, но её не существует. Вторая — public, и она существует по умолчанию. В ней и будет создана таблица.

### Таблицы: виды, дистрибуция и сжатие

Мы разобрались, что такое база данных и схема, а теперь пойдём к таблицам. 
Какие полезные свойства могут быть у таблиц в Greenplum:
 - ⭐️ Дистрибуция
 - ⭐️ Тип хранения
 - ⭐️ Ориентация
 - ⭐️ Сжатие
 - ⭐️ Партицирование
   
А ещё бывают временные, нежурналированные и внешние таблицы.

### Дистрибуция
В основе концепции MPP лежит идея о равномерном разделении данных между сегментами, чтобы распараллелить и ускорить их обработку. В зависимости от СУБД такое распределение данных может называться шардированием или дистрибуцией. **В Greenplum используется термин дистрибуция.**

💡 **Сегмент Greenplum** —  это инстанс модифицированной СУБД PostgreSQL, единица шардирования данных и параллелизации работы. Сегменты располагаются на сегмент-хостах, и их может быть несколько на хост.

Для эффективной работы MPP СУБД данные должны быть равномерно распределены между сегментами: так каждый сегмент обработает свою часть данных и быстро отдаст результат. 

Часть данных, находящаяся на одном из сегментных инстансов PostgreSQL, называется сегмент, или шард. В зависимости от контекста, сегмент — это и **инстанс PostgreSQL в составе кластера, и часть данных таблицы, хранящаяся в этом инстансе.**

<img src='images/00014.png' width=300><br>
Существует несколько типов дистрибуции данных, рассмотрим их ниже.

В Greenplum нет возможности хранить несколько копий части данных (шарда) на одном сегменте. Поэтому у коэффициента репликации может быть два значения: 
— 1, если кластер без зеркалирования;
— 2 при использовании зеркалирования сегментов.


### Типы дистрибуции в Greenplum
#### ⭐️ Дистрибуция по ключу (DISTRIBUTED BY)
Ключ дистрибуции может состоять из одного или нескольких полей таблицы. У ключа должна быть высокая **кардинальность** — большое количество уникальных значений для как можно более равномерного распределения данных по сегментам. В качестве ключа дистрибуции выгодно выбирать ключ, по которому в дальнейшем будут объединяться таблицы (join).

💡 В составной ключ дистрибуции рекомендуется добавлять не более трёх полей.
```
Пример создания таблицы с ключом распределения из нескольких полей:

CREATE TABLE test_schema.company
(
  id UUID,
  inn BIGINT,
  company_name TEXT,
  utc_valid_from_dttm TIMESTAMP WITHOUT TIME ZONE
)
DISTRIBUTED BY (id, inn);
```
#### ⭐️ Дистрибуция случайным образом (DISTRIBUTED RANDOMLY)
Если не удаётся подобрать поля для хорошего ключа распределения, можно использовать дистрибуцию случайным образом. Строки будут распределены по сегментам равномерно.
```
Пример создания таблицы со случайным распределением:

CREATE TABLE test_schema.company
(
    id UUID,
    inn BIGINT,
    company_name TEXT,
    utc_valid_from_dttm TIMESTAMP WITHOUT TIME ZONE
)
DISTRIBUTED RANDOMLY;
```
#### ⭐️ Реплицирование таблицы по всем сегментам (DISTRIBUTED REPLICATED)
Словари и небольшие таблицы можно хранить как полную копию на каждом сегменте кластера. В ряде случаев это помогает ускорить запросы с использованием этих таблиц.

❗️ Объём места, занятый на кластере таблицей, увеличивается кратно количеству сегментов, то есть в десятки или сотни раз. Мы не рекомендуем выбирать такой способ распределения для больших таблиц.
```
Пример создания таблицы, реплицированной на все сегменты:

CREATE TABLE test_schema.department
(
        id UUID,
        dept_name TEXT,
        utc_valid_from_dttm TIMESTAMP WITHOUT TIME ZONE
)
DISTRIBUTED REPLICATED;
```
**Почему выбор ключа дистрибуции важен**

Неоптимально выбранный ключ дистрибуции может привести к перекосам: на одном сегменте — несколько десятков строк, а на другом — сотни или тысячи. Это приведёт к более длительному времени обработки на одних сегментах, в то время как другие будут простаивать.

**💡 Скорость выполнения запроса равна скорости обработки данных на самом медленном сегменте.**

Если в выражении CREATE TABLE не указать ключ дистрибуции, то в созданной таблице будет выбран метод дистрибуции **по умолчанию** и выведется предупреждение.
```
Пример создания таблицы с распределением по умолчанию: 

CREATE TABLE test_schema.company
(
  id UUID,
  inn BIGINT,
  company_name TEXT,
  utc_valid_from_dttm TIMESTAMP WITHOUT TIME ZONE
); -- ключ дистрибуции не указан!

После выполнения этой команды Greenplum выдаст предупреждение:
"NOTICE:  Table doesn't have 'DISTRIBUTED BY' clause -- Using column named 'id' as the Greenplum Database data distribution key for this table."
"HINT:  The 'DISTRIBUTED BY' clause determines the distribution of datа
```
❗️ Всегда явно указывайте дистрибуцию, иначе кластер сам выбирает поле дистрибуции (как правило, это первое поле в таблице) и распределяет таблицу по сегментам по этому полю. Таблица может распределиться неравномерно, и будет перекос.

Ключ дистрибуции таблицы можно менять с помощью выражения `ALTER TABLE`:
```
ALTER TABLE test_schema.company SET DISTRIBUTED BY (inn);
```
Если таблица не пустая, необходимо провести физическое перераспределение данных по сегментам. Для этого нужно выполнить команду:

`ALTER TABLE test_schema.company SET WITH (REORGANIZE=true); `

❗️ Для большой таблицы выполнение команды REORGANIZE может занять длительное время, так как Greenplum будет заново создавать файлы таблицы на диске с данными, распределёнными по новой политике дистрибуции.

А теперь познакомимся с опциями  физического размещения данных в таблицах Greenplum. И первая из них — **способ организации файлов таблицы на диске**: HEAP и APPEND-OPTIMIZED.
### HEAP- и APPEND-OPTIMIZED-таблицы
#### HEAP-таблицы
Простая внутренняя структура хранения, перекочевавшая из PostgreSQL. На каждом сегменте Greenplum записи таблицы складываются последовательно в один файл.

 - 💡 HEAP-таблицы создают наименьшее количество файлов в файловой системе.
 - ❗️ Количество файлов под HEAP-таблицами может увеличиваться при превышении размера файла 1 ГБ или при хранении в таблице типов данных переменной длины (например, varchar).
 - 💡 HEAP-таблицы поддерживают только построчно-ориентированное хранение данных.
 - ❗️ Сжатие HEAP-таблиц недоступно.

##### Когда использовать HEAP-таблицы:
 - ⭐️ Для случаев, когда данные в таблице часто изменяются с помощью операций `DELETE, INSERT и UPDATE`.
 - ⭐️ Для таблиц малого размера — например, до гигабайта.
##### Синтаксис HEAP-таблиц
 - 💡 Если в CREATE TABLE не указывать никаких опций хранения для таблицы, по умолчанию создаются таблицы типа HEAP.
```
Пример создания HEAP-таблицы:

CREATE TABLE test 
(
    id INT,
    content TEXT,
    VALUE NUMERIC(15,2)
)  
DISTRIBUTED BY (id);
```

### APPEND-OPTIMIZED-таблицы
APPEND-OPTIMIZED-таблицы нужны для сжатия данных. У них более сложная внутренняя структура: таблицы содержат ссылки на файлы, в которых хранятся большие блоки данных. У каждой таблицы есть внутренний индекс, обеспечивающий её работу. Это индекс по системным полям, и он не используется в запросах.

Каждая APPEND-OPTIMIZED-таблица создаёт несколько системных объектов в Greenplum и несколько файлов на диске.

<img src='images/00015.png' width=500><br>
💡 Главное достоинство AO-таблиц — возможность хранить данные в сжатом виде. Если сжатие не нужно, лучше выбрать HEAP-таблицу.

##### Когда использовать АО-таблицы:
 - ⭐️ Только для средних и больших массивов данных. В противном случае накладные расходы на хранение таблицы превышают выгоду от сжатия данных.
 - ⭐️ При использовании массовой вставки данных. 
 - ⭐️ Для аналитической обработки больших объёмов данных. 

AO-таблицы создаются при указании `appendoptimized=TRUE в блоке with()`. На практике для AO-таблиц  всегда применяются опции ориентации или сжатия, которые тоже указываются в блоке with(), поэтому полный синтаксис для создания AO-таблиц покажем позже в этом уроке. 
#### Ориентация
💡 APPEND-OPTIMIZED-таблицы поддерживают как построчно-, так и поколоночно-ориентированное хранение данных.
 - Построчная ориентация: в файле данных записи хранятся целиком и последовательно.
 - Поколоночная ориентация: каждая колонка таблицы хранится в отдельном файле. Это приводит к резкому увеличению количества файлов, в которых хранятся данные. AO-таблицы с поколоночной ориентацией значительно сокращают затраты на чтение, когда запрос обращается только к части столбцов из всего набора колонок таблицы. Такие таблицы лучше сжимаются.
```
Ориентация задаётся в блоке with()следующим выражением:

orientation=ROW     -- Построчная ориентация
-- или
orientation=COLUMN  -- Поколоночная ориентация
```
При отсутствии опции orientation по умолчанию принимается значение **row — построчная ориентация**.
```
Пример создания AO-таблицы с поколоночной ориентацией:

CREATE TABLE test 
(
    id SERIAL,
    content TEXT,
    VALUE NUMERIC(15,2)
)  
DISTRIBUTED RANDOMLY
WITH (appendoptimized=TRUE, orientation=COLUMN);
```

 
#### Сжатие
Greenplum позволяет сжимать данные в таблицах, используя различные алгоритмы сжатия. 

❗️ Сжатие поддерживается только для AO-таблиц `(appendoptimized=TRUE)`. Используйте их, если требуется сжимать данные. А если нет — выбирайте HEAP-таблицы.
```
Опция сжатия при создании таблицы указывается в блоке with():

compresstype={ZLIB|ZSTD|RLE_TYPE},
compresslevel={1-19}
```
Сжатие можно задать только при создании таблицы. Вид таблицы, алгоритм и уровень сжатия невозможно поменять операцией ALTER TABLE. Их можно изменить, только ***пересоздав таблицу**.

💡 AO-таблицы с поколоночным хранением данных могут иметь колонки с различающимися алгоритмами и степенями сжатия.

❗️ Сжатие позволяет сэкономить дисковое пространство: это происходит за счёт использования CPU. Поэтому запись в сжатую таблицу может быть медленнее, чем в HEAP, особенно на быстрых дисках. Чтение из сжатой таблицы использует CPU менее интенсивно, чем запись, и не зависит от алгоритма сжатия.

 Виды сжатия и их особенности
 
 <img src='images/00016.png' width=500><br>
💡 На практике для типов сжатия **zstd и zlib** стоит выбирать степень сжатия в диапазоне **1–2**, а для **rle_type — 2**. Они дают максимальную производительность при приемлемом уровне сжатия.
```
Пример создания сжатой таблицы:

CREATE TABLE test_schema.company
(
    id                      UUID,
    inn                     BIGNT,
    company_name            TEXT,
    utc_valid_from_dttm     TIMESTAMP
)
WITH (appendoptimized = TRUE, compresstype = zstd, compresslevel = 2)
DISTRIBUTED BY (id);
```
Подытожим: тип таблицы, алгоритм и степень сжатия задаются в секции with() в DDL (Data Definition Language — операции, связанные с созданием и изменением объектов БД) таблицы при её создании и в дальнейшем не могут быть изменены.

 <img src='images/00017.png' width=600><br>
 Наиболее часто встречающиеся параметры типа таблиц, ориентации и сжатия

### Временные таблицы
Greenplum поддерживает создание временных таблиц для сохранения промежуточных результатов.

💡 **Временные таблицы (TEMPORARY) **— живут в рамках одной сессии и автоматически уничтожаются при завершении сессии.

Сессия начинается подключением конкретного пользователя к конкретной БД и завершается отключением от неё. В рамках сессии может быть выполнено любое количество запросов, DML-операций и DDL-операций. 
```
Пример создания временной таблицы:

CREATE TEMPORARY TABLE my_temp_table (
    id SERIAL,
    content TEXT,
    value NUMERIC(15,2)
)
WITH (appendoptimized=TRUE, orientation=row, compresstype=zstd, compresslevel=1)
DISTRIBUTED BY (id);
```
Имена временных таблиц всегда пишутся без указания схемы. Временные таблицы создаются в системной схеме pg_temp_<номер сессии>, которая не присутствует явно в search_path, но участвует в пути поиска при обращении к таблицам. Эта схема удаляется по завершении сессии.

Временные таблицы, так же как и обычные, могут быть HEAP или AO, поддерживать параметры хранения (сжатие и колоночно-ориентированное хранение) и иметь ключ распределения. Все типы полей обычных таблиц поддерживаются и во временных таблицах. 

Для временных таблиц можно задать поведение в конце транзакции с помощью опции `ON COMMIT`:
 - `ON COMMIT DELETE ROWS` — все строки во временной таблице будут удалены в конце каждого блока транзакции.
 - `ON COMMIT DROP` — временная таблица будет удалена в конце блока текущей транзакции.
```
Пример указания ON COMMIT DROP:

CREATE TEMPORARY TABLE my_temp_table
(
    id SERIAL,
    content TEXT,
    VALUE NUMERIC(15,2)
)
WITH (appendoptimized=TRUE, orientation=row, compresstype=zstd, compresslevel=1)
DISTRIBUTED BY (id)
ON COMMIT DROP;
```
**Когда использовать временные таблицы:**

⭐️ Когда нужно сохранить данные промежуточного расчёта, чтобы использовать их дальше для получения окончательного результата. <br>
Например, если для получения окончательной витрины нужно соединить несколько десятков таблиц, разумно будет разбить такую операцию на несколько блоков, сохраняя результат каждого блока во временную таблицу и используя её в следующем блоке. Так уменьшится сложность запроса и улучшится читаемость кода.

💡 Не забывайте указывать дистрибуцию во временных таблицах.

### Нежурналируемые таблицы

Данные в таблицах Greenplum зеркалируются на сегментах-зеркалах. Записи попадают на зеркала через журнал изменений базы данных. Цена зеркалирования — двойной объём операций записи на диск и двойной объём места.

Когда такая надёжность не требуется, можно использовать нежурналируемые таблицы, которые
 - не используют WAL;
 - не реплицируются на сегментах-зеркалах.

При записи в такую таблицу строки пишутся напрямую в файл данных на первичном сегменте, и это даёт выигрыш в скорости записи. При чтении нежурналируемые таблицы не дают преимущества над обычными.

💡 Если хотя бы один сегмент Greenplum выйдет из строя, все нежурналируемые таблицы станут недоступны.

#### Преимущества и недостатки нежурналируемых таблиц:
 - ✅ Скорость работы с такой таблицей выше, чем с обычной, так как уменьшается количество операций записи на диск. 
 - ❌ Уменьшается надёжность хранения данных.

❗️ Данные из нежурналируемых таблиц не сохраняются в резервных копиях Managed Greenplum, поскольку резервное копирование в нём использует WAL. При восстановлении из резервной копии нежурналируемая таблица будет создана пустой.


Первое правило хранения данных — никому не рассказывай про данные и храни их надёжно. А ещё лучше заранее думай, действительно ли ты готов рискнуть своими данными, храня их в нежурналируемых таблицах. 

Синтаксис для нежурналируемых таблиц такой же, как и для обычных. Единственное отличие — указание слова `UNLOGGED` при создании таблицы:
```
CREATE UNLOGGED TABLE my_unlogged_table
(
    id SERIAL,
    content TEXT,
    VALUE NUMERIC(15,2)
)
WITH (appendoptimized=true, orientation=row)
DISTRIBUTED BY (id);
```
Опции сжатия и распределения при создании нежурналируемой таблицы доступны, как и для обычных таблиц.

💡 Нежурналируемые таблицы можно использовать для хранения данных, потеря которых не критична. Например временные таблицы в ETL процессах можно делать нежурналируемыми.
### Внешние таблицы
Внешние таблицы не хранят в себе данные, но описывают формат и путь к данным, находящимся во внешних источниках, например файлах или других СУБД.

💡 Внешние таблицы — один из эффективных способов параллельной загрузки данных в Greenplum или выгрузки из него.

Внешние таблицы создаются с помощью команды `CREATE EXTERNAL TABLE` .

Подробнее о внешних таблицах расскажем далее в курсе.

### ✍️ Практическое задание 2: алгоритмы сжатия
 - 1️⃣ Создайте таблицу с произвольными полями и сжатием алгоритмом zstd уровня 2.
 - 2️⃣ Создайте аналогичную таблицу без сжатия.
 - 3️⃣ Заполните обе таблицы одинаковыми данными (100 000 или более строк). Используйте функцию generate_series(start BIGINT, end BIGINT) для генерации множества строк.
 - 4️⃣ Сравните размер таблиц при помощи запроса SELECT pg_size_pretty(pg_total_relation_size('<имя таблицы>'));.

🤝 Правильное решение

Выполните код:

```
--1 Создайте таблицу с произвольными полями и сжатием алгоритмом zstd уровня 2
CREATE TABLE compression_test_ao_zstd2 (n bigint, s text) 
WITH (appendoptimized=true, compresstype=zstd, compresslevel=2) 
DISTRIBUTED RANDOMLY;
--2 Создайте аналогичную таблицу без сжатия.
CREATE TABLE compression_test_heap (LIKE compression_test_ao_zstd2)
DISTRIBUTED RANDOMLY;
--3 Заполните обе таблицы одинаковыми данными
INSERT INTO compression_test_ao_zstd2 
SELECT g, 'Line '||g FROM generate_series(1,100000) AS g;

INSERT INTO compression_test_heap 
SELECT g, 'Line '||g FROM generate_series(1,100000) AS g;

--4 Сравните размер таблиц при помощи запроса
SELECT 
  pg_size_pretty(pg_total_relation_size('compression_test_ao_zstd2')) AS compressed_size,
  pg_size_pretty(pg_total_relation_size('compression_test_heap')) AS uncompressed_size
;

Результат:

 compressed_size | uncompressed_size 
-----------------+-------------------
 1368 kB         | 5984 kB
 
```

тест
1. Что такое дистрибуция?
   - Распределение данных таблицы между сегментами в соответствии с выбранной политикой.
2. Что вы выберете, чтобы создать построчно-ориентированную таблицу и сжимать данные в ней?  
   - Таблицу AO и сжатие zstd.
3. Выберите два верных свойства временных таблиц.  
   - Создаются и существуют в рамках сессии или транзакции. и - Могут хранить данные в сжатом виде.